# Transformación con petl

![Actividades ETL](./img/etl_actividades.svg)

## Librería petl

- [petl](https://petl.readthedocs.io/en/stable/index.html) es un paquete de propósito general para la extracción, transformación y carga de tablas de datos.
- Diseñado especialmente para trabajar __datos desconocidos, heterogéneos y/o de calidad mixta__.
- __No tiene dependencias__ que no sean módulos propios del nucleo de Python.
- Las _pipelines_ (transformaciones de flujo de datos en un proceso comprendido por varias fases secuenciales) hacen __uso mínimo de la memoria del sistema__ y pueden __escalar a millones de filas si la velocidad no es una prioridad__.
- En general, __los flujos de trabajo no se ejecutan hasta que se soliciten los datos__.
- Soporta programación funcional y orientada a objetos.
-  [Otras alternativas para diseñar procesos ETL](https://petl.readthedocs.io/en/stable/related_work.html)

[Instalación de la librería petl](https://petl.readthedocs.io/en/stable/intro.html#installation)

## Conversión de valores

La función `.convert()` permite __convertir valores invocando funciones__ y los campos afectos como un diccionario. 

Por ejemplo, valores de una columnas desde `str` a `float`:

In [12]:
import petl as etl

niveles = etl.fromxlsx('./dataset/niveles.xlsx')
niveles.look()

+--------------+-----------+
| Fecha        | Nivel (m) |
+==============+===========+
| '2012-01-23' |     64.59 |
+--------------+-----------+
| '2012-05-23' |     64.79 |
+--------------+-----------+
| '2012-07-30' |     64.35 |
+--------------+-----------+
| '2012-09-10' |     64.77 |
+--------------+-----------+
| '2012-11-15' |     68.05 |
+--------------+-----------+
...

In [19]:
type(niveles[1][1])

float

In [17]:
n_niveles = etl.convert(niveles, {'Nivel (m)': int})
n_niveles.look()

+--------------+-----------+
| Fecha        | Nivel (m) |
+==============+===========+
| '2012-01-23' |        64 |
+--------------+-----------+
| '2012-05-23' |        64 |
+--------------+-----------+
| '2012-07-30' |        64 |
+--------------+-----------+
| '2012-09-10' |        64 |
+--------------+-----------+
| '2012-11-15' |        68 |
+--------------+-----------+
...

In [21]:
type(n_niveles[1][1])

int

También se permite la __sustitución de valores__ invocando los valores como un diccionario cuyo conjunto `key:value` corresponde al origen y destino, respectivamente.

In [42]:
tabl.look()

+-----+-------+-------+
| num | lower | upper |
+=====+=======+=======+
|   0 | 'a'   | 'A'   |
+-----+-------+-------+
|   1 | 'b'   | 'B'   |
+-----+-------+-------+
|   2 | 'c'   | 'C'   |
+-----+-------+-------+
|   3 | 'a'   | None  |
+-----+-------+-------+

In [25]:
empleados = etl.fromjson('./dataset/empleados.json')
empleados_1 = etl.convert(empleados, 'ciudad', {'Santiago': 'Metropolitana', 'Concepción': 'Biobio'})
empleados_1
# etl.look(tabl6)

nombre,segundo_nombre,edad,ciudad,casado
Juan,None,30,Metropolitana,True
Sandra,Ariel,24,Biobio,False


Otras [alternativas de conversión de valores](https://petl.readthedocs.io/en/stable/transform.html#converting-values):
- Conversión pasando el método y sus parámetros.
- Conversión condicional.
- Conversión accediendo a datos de la misma fila.
- Conversión de todo a número.
- Conversión por interpolación.
- ...

## Filtros y búsqueda

El segundo argumento de la función `.select()` puede ser una expresión con forma de condición, expresado como un literal de tipo `str`.

In [30]:
niveles_2012 = etl.select(niveles, "{Fecha}>'2012' and {Fecha}<'2013'")
niveles_2012.lookall()

+--------------+-----------+
| Fecha        | Nivel (m) |
+==============+===========+
| '2012-01-23' |     64.59 |
+--------------+-----------+
| '2012-05-23' |     64.79 |
+--------------+-----------+
| '2012-07-30' |     64.35 |
+--------------+-----------+
| '2012-09-10' |     64.77 |
+--------------+-----------+
| '2012-11-15' |     68.05 |
+--------------+-----------+

In [31]:
filtro_73 = etl.select(niveles, "{Nivel (m)} != 73.33")
filtro_73.lookall()

+--------------+-----------+
| Fecha        | Nivel (m) |
+==============+===========+
| '2012-01-23' |     64.59 |
+--------------+-----------+
| '2012-05-23' |     64.79 |
+--------------+-----------+
| '2012-07-30' |     64.35 |
+--------------+-----------+
| '2012-09-10' |     64.77 |
+--------------+-----------+
| '2012-11-15' |     68.05 |
+--------------+-----------+
| '2013-05-16' |     75.19 |
+--------------+-----------+
| '2013-07-29' | None      |
+--------------+-----------+
| '2013-09-11' |     72.68 |
+--------------+-----------+
| '2014-03-27' |     85.12 |
+--------------+-----------+
| '2014-05-22' |     83.77 |
+--------------+-----------+
| '2014-07-25' |     86.19 |
+--------------+-----------+
| '2014-09-30' |     88.57 |
+--------------+-----------+
| '2014-11-25' | None      |
+--------------+-----------+
| '2015-05-22' |    102.14 |
+--------------+-----------+

Es posible __mapear campos de valores de una tabla en un diccionario__. La función `.facet()` recibe como segundo argumento la columna que será mapeada como claves del diccionario.

In [32]:
empleados.look()

+----------+----------------+------+--------------+--------+
| nombre   | segundo_nombre | edad | ciudad       | casado |
+==========+================+======+==============+========+
| 'Juan'   | None           |   30 | 'Santiago'   | True   |
+----------+----------------+------+--------------+--------+
| 'Sandra' | 'Ariel'        |   24 | 'Concepción' | False  |
+----------+----------------+------+--------------+--------+

In [34]:
nombre = etl.facet(empleados, 'nombre')
nombre.keys()

dict_keys(['Juan', 'Sandra'])

In [36]:
nombre['Juan'].look()

+--------+----------------+------+------------+--------+
| nombre | segundo_nombre | edad | ciudad     | casado |
+========+================+======+============+========+
| 'Juan' | None           |   30 | 'Santiago' | True   |
+--------+----------------+------+------------+--------+

## Uso de expresiones regulares

A partir de [expresiones regulares](https://docs.python.org/3/library/re.html#regular-expression-syntax) se realiza una busqueda de valores y __retorna las filas que coinciden con un patrón__.

In [38]:
import pandas as pd

url = 'http://www.sismologia.cl/ultimos_sismos.html'
tablas = pd.read_html(url)
sismos_df = tablas[0]
sismos = etl.fromdataframe(sismos_df)

etl.cut(sismos, 'Referencia Geográfica').look()

+------------------------------+
| Referencia Geográfica        |
+==============================+
| '50 km al O de Taltal'       |
+------------------------------+
| '36 km al NO de María Elena' |
+------------------------------+
| '69 km al NE de Calama'      |
+------------------------------+
| '13 km al SE de Cuya'        |
+------------------------------+
| '30 km al NO de Canela Baja' |
+------------------------------+
...

En __cualquier lugar de un registro__ (fila):

In [44]:
sismos

Fecha Local,Fecha UTC,Latitud,Longitud,Profundidad [Km],Magnitud,Referencia Geográfica
2021-10-20 01:09:02,2021-10-20 04:09:02,-25.53,-70.963,48.0,3.2 Ml,50 km al O de Taltal
2021-10-19 21:49:05,2021-10-20 00:49:05,-22.06,-69.826,59.5,2.5 Ml,36 km al NO de María Elena
2021-10-19 21:22:31,2021-10-20 00:22:31,-21.902,-68.652,125.0,3.5 Ml,69 km al NE de Calama
2021-10-19 20:37:37,2021-10-19 23:37:37,-19.261,-70.109,54.3,2.7 Ml,13 km al SE de Cuya
2021-10-19 20:28:48,2021-10-19 23:28:48,-31.268,-71.733,34.9,3.2 Ml,30 km al NO de Canela Baja


In [46]:
sismos_18 = etl.search(sismos, '18')
sismos_18

Fecha Local,Fecha UTC,Latitud,Longitud,Profundidad [Km],Magnitud,Referencia Geográfica
2021-10-19 18:01:05,2021-10-19 21:01:05,-18.963,-69.28,112.3,3.6 Ml,42 km al NE de Camiña
2021-10-19 15:39:00,2021-10-19 18:39:00,-20.329,-69.678,64.9,2.6 Ml,14 km al SE de Pozo Almonte
2021-10-19 15:37:57,2021-10-19 18:37:57,-27.754,-71.091,36.8,4.1 Ml,80 km al N de Huasco
2021-10-19 12:43:41,2021-10-19 15:43:41,-18.603,-71.103,18.7,3.1 Ml,84 km al O de Arica


Dentro de __un campo específico__:

In [47]:
sismos_collahuasi = etl.search(sismos, 'Referencia Geográfica', 'Collahuasi')
sismos_collahuasi.cut('Magnitud', 'Referencia Geográfica').look()

+----------+----------------------------------+
| Magnitud | Referencia Geográfica            |
+==========+==================================+
| '2.5 Ml' | '27 km al NO de Mina Collahuasi' |
+----------+----------------------------------+

### Ordenamiento

Por defecto, la función `.sort()` realiza sobre una tabla un ordenamiento léxico:

In [49]:
catalogo = etl.fromxml('./dataset/cd_catalog.xml', 'CD', 
                       {'cia': 'COMPANY', 'artista': 'ARTIST', 'pais': 'COUNTRY'})
catalogo

artista,cia,pais
Bob Dylan,Columbia,USA
Bonnie Tyler,CBS Records,UK
Dolly Parton,RCA,USA
Gary Moore,Virgin records,UK
Eros Ramazzotti,BMG,EU


In [50]:
catalogo_ordenado = etl.sort(catalogo)
etl.lookall(catalogo_ordenado)

+---------------------+------------------+----------+
| artista             | cia              | pais     |
+=====================+==================+==========+
| 'Andrea Bocelli'    | 'Polydor'        | 'EU'     |
+---------------------+------------------+----------+
| 'Bee Gees'          | 'Polydor'        | 'UK'     |
+---------------------+------------------+----------+
| 'Bob Dylan'         | 'Columbia'       | 'USA'    |
+---------------------+------------------+----------+
| 'Bonnie Tyler'      | 'CBS Records'    | 'UK'     |
+---------------------+------------------+----------+
| 'Cat Stevens'       | 'Island'         | 'UK'     |
+---------------------+------------------+----------+
| 'Dolly Parton'      | 'RCA'            | 'USA'    |
+---------------------+------------------+----------+
| 'Dr.Hook'           | 'CBS'            | 'UK'     |
+---------------------+------------------+----------+
| 'Eros Ramazzotti'   | 'BMG'            | 'EU'     |
+---------------------+-----

Ingresando como argumento el nombre de __uno o más campos__:
- El parámetro `reverse=True` permite ejecutar un ordenamiento en sentido contrario.

In [52]:
sismos_ordenados = etl.sort(sismos, key=['Magnitud'], reverse=True)
sismos_ordenados.cut('Magnitud', 'Referencia Geográfica').look()

+----------+----------------------------+
| Magnitud | Referencia Geográfica      |
+==========+============================+
| '4.1 Ml' | '80 km al N de Huasco'     |
+----------+----------------------------+
| '3.6 Ml' | '42 km al NE de Camiña'    |
+----------+----------------------------+
| '3.6 Ml' | '58 km al SO de Los Vilos' |
+----------+----------------------------+
| '3.5 Ml' | '69 km al NE de Calama'    |
+----------+----------------------------+
| '3.2 Ml' | '50 km al O de Taltal'     |
+----------+----------------------------+
...

También es posible __comprobar el ordenamiento de una tabla__:

In [53]:
etl.issorted(sismos_ordenados, key='Referencia Geográfica')

False

Otras operaciones de transformación:
- `mergesort()`
- `join()`
- `unjoin()` (split)
- `duplicates()`
- `unique()`
- `aggregate()`
- `reshape()`
- `transpose()`
- `pivot()`
- `fill...()`


## Extras

Encontrar __mínimo y máximo__:

In [63]:
mn, mx = etl.limits(sismos_collahuasi, 'Magnitud')
print('Máx: {}\nMín: {}'.format(mx, mn))

Máx: 4.2 Mw
Mín: 3.0 Ml


Algunos __estadísticos básicos__:

In [54]:
estadisticos_niveles = etl.stats(niveles, 'Nivel (m)')
estadisticos_niveles

stats(count=13, errors=2, sum=993.5399999999998, min=64.35, max=102.14, mean=76.42615384615385, pvariance=129.69419289940822, pstdev=11.388335826599434)

__Contar valores__:

In [55]:
etl.valuecounts(catalogo, 'cia').lookall()

+------------------+-------+----------------------+
| cia              | count | frequency            |
+==================+=======+======================+
| 'Polydor'        |     3 |  0.11538461538461539 |
+------------------+-------+----------------------+
| 'Columbia'       |     2 |  0.07692307692307693 |
+------------------+-------+----------------------+
| 'Atlantic'       |     2 |  0.07692307692307693 |
+------------------+-------+----------------------+
| 'CBS Records'    |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'RCA'            |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'Virgin records' |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'BMG'            |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'CBS'            |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'Pickwick'